# Speeding Up with Cython

We should be able to use Cython to speed up our functions.

We should also be able to work in 8-bit space - we are not worried about being exact.

The only library we are using is numpy so we need to work out how to use Cython with Numpy. Here is an article - https://blog.paperspace.com/faster-numpy-array-processing-ndarray-cython/.

## Core Classes

We have three core classes:
* Covariance
* Power Iterator
* VPU

### Covariance

We'll start with the ZeroMeanCovarianceUnit.

Data:
* Size - small integer 8-bit unsigned.
* Count - we could limit to 255? (i.e. 8-bit unsigned) At the moment just runs upwards so would be large int.
* Square Sum - Numpy L * L 2D array
    * The data points x are 1D arrays of -1 to 1. 
    * x dot x.T results in an L by L matrix. Each i, j th entry is x_i * x_j. So min is -1 and max is 1. So x dot x.T is a 2D array which can be 3 bit (or two binary matrixes - one positive, one negative or one with abs and one with sign information).
    * Without scaling the square sum is a 2D integer array where the maximum integer is equal to the number of iterations.
    * If there are 256 iterations we go from 8-bit to 16-bit.
    * The covariance matrix is the square sum divided by the count. If we have 255 iterations each value could be mulitples of 1/255 - if we have 145 iterations it will be multiples of 1/145.
    
    
Can we have an assumed division in our covariance matrix and have it as modulo bit-length (e.g. 255)? NEED TO DEVELOP THIS. We have iteration batches of bit_length (e.g. 255). When we reach count = 256 we bit shift right (divide by 2) and add. Or if count is an 8-bit int when count = 255 we run and then reset count to 0. Hence, we have a fraction resolution set by bit_depth.

This will limit though to adapting to recent signals - we give our old signal and our new signal equal weight - but if our recent signal is very different but we have a normally stable variation (e.g. staring at a light) this will cause our covariance matrix to change. 

To cope we can change the right shift amounts. E.g. more heavy right shifts decrease our resolution of our updates but increase our resolution of our historic data. Weighting each.

A good question is do we need a 16-bit resolution or is a 8-bit resolution functional?

*Is this related to our scale system?* We have different covariance matrices representing different scales and then when we want the covariance matrix we combine and divide? Only need 10 log2 matrices. Buffers over time.

It is related to our time buffers and time series. The reconstruction is our covariance. See 2020-02-21 - Playing with Time Series Configurations.

Algorithm:
* On first run calculate SS as normal (or minus 0 - maybe minus 128)
* At iteration = bit_depth:
    * Add strata (our old buffer).
    * PBT the values between 0 and 255 to create binary version.
    * Add binary version to rolling sum for new strata.
    * If iteration = 255^i send estimate back and subtract instead of 128?
    
This is very very similar to our binary estimation case and reconstruction of predicted input. Coincidence?!

255^8 at 1Hz is 566 535 009 millennia. Even at 1MHz this would still be several millennia. (255^4)s in years is 133 years. CPU is 1200MHz. 1.2 * 10^9. for an approximate result, divide the time value by 3.154e+7

In [4]:
((255**8)/(1.2*(10**9)))/(3.154*(10**7))

472.36586735924993

In [8]:
(((2**8)**8)/(1.2*(10**9)))/(3.154*(10**7))

487.39019429585585

In [13]:
((2**64)/(1.2*(10**9)))/(3.154*(10**7))

487.39019429585585

In [5]:
((255**7)/(1.2*(10**9)))/(3.154*(10**7))

1.8524151661147055

So 8 sets of buffers would give us enough for 472 years at full CPU cycles.

In [1]:
512/4

128.0

Can we initialise the square sums as 128? Hence, we can subtract them? And change them?

Above 255 = 2^8 where 8 = bit depth so 8 stages at 255 is the same as 2^16. So the stages are 64/bit_depth with max 64.

In [15]:
bd = 68
print(bd)
if bd > 64: bd = 64
print(bd)
bd = 62
print(bd)
if bd > 64: bd = 64
print(bd)

68
64
62
62


Init if not 8 bit.
```
def __init__(self, size, bit_depth=8):
        """Initialise.

        Args:
            size: integer setting the 1D size of an input.
            bit_depth: number of bits to store representations (multiple of 8) - max 64
        """
        self.size = size
        self.count = 0
        # Set bit_depth modulo 8
        if bit_depth > 64: bit_depth = 64
        if bit_depth < 8: bit_depth = 8
        stages = 64//bit_depth
        # Initialise Square Sum
        self.square_sums = np.ones(shape=(size, size, stages))
```

In [20]:
2**8

256

Does this only have benefit if we actually start to think that our representation space is limited?

When using Cython its often better to define lists as 1D numpy arrays. So let's start doing that.

```cdef np.array[double, dim=1] x_array```

In [22]:
import numpy as np
np.zeros(8)

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [28]:
# Can we do 2D PBT?
rand_vals_1 = np.random.randint(255, size=(4,4))
rand_vals_2 = np.random.randint(255, size=(4,4))
binary_values = np.where(rand_vals_1 > rand_vals_2, 1, 0)
print(rand_vals_1, rand_vals_2, binary_values)

[[127 183 235 207]
 [ 82 234 230  90]
 [178 115 193 226]
 [250 142 173 184]] [[171 228  30 215]
 [233 152 128 218]
 [141   9 252 124]
 [200  52 248  28]] [[0 0 1 0]
 [0 1 1 0]
 [1 1 0 1]
 [1 1 0 1]]


In [29]:
 binary_values[:, 0] = 128; binary_values

array([[128,   0,   1,   0],
       [128,   1,   1,   0],
       [128,   1,   0,   1],
       [128,   1,   0,   1]])

In [30]:
127//2

63

When ensuring that we remain within signed 8-bits integers we can divide by 2 before subtracting. But like our other residuals most of the energy is around the mean. So clipping maybe better. We thus clip at -64 and +63 subtract then times by 2.
```
# Right bit shift the current sum and the next stage sum
            # BUT LIKE OUR RESIDUALS WE MIGHT FIND ITS BETTER TO CLIP THAN DIVIDE
            current_stage = self.square_sum[:, :, 0]//2
            next_stage = self.square_sum[:, :, 1]//2
```

Clipping will only work if our input signal is already zero mean - 

One problem is our data may be - or +. Hence, our square sum may have +1 or -1 terms at each iteration. But if we get 254 [1, 1] vectors our square sum will be 254 (or [1, -1] will get you [[254, -254], [-254, 254]]).

In [97]:
test_array = np.asarray([[1],[-1]]); print(test_array.shape)

test_sum = np.zeros(shape=(2,2))
for _ in range(0, 254):
    test_sum += np.dot(test_array, test_array.T)

print(test_sum, test_sum.astype(np.int8))

(2, 1)
[[ 254. -254.]
 [-254.  254.]] [[-2  2]
 [ 2 -2]]


In [98]:
test_array = np.asarray([[1],[1]]); print(test_array.shape)

test_sum = np.zeros(shape=(2,2))
for _ in range(0, 254):
    test_sum += np.dot(test_array, test_array.T)

print(test_sum, test_sum.astype(np.int8))

(2, 1)
[[254. 254.]
 [254. 254.]] [[-2 -2]
 [-2 -2]]


In [99]:
test_array = np.asarray([[-1],[-1]]); print(test_array.shape)

test_sum = np.zeros(shape=(2,2))
for _ in range(0, 254):
    test_sum += np.dot(test_array, test_array.T)

print(test_sum, test_sum.astype(np.int8))

(2, 1)
[[254. 254.]
 [254. 254.]] [[-2 -2]
 [-2 -2]]


We can just iterate 127 times.

In [100]:
test_array = np.asarray([[1],[-1]]); print(test_array.shape)

test_sum = np.zeros(shape=(2,2))
for _ in range(0, 127):
    test_sum += np.dot(test_array, test_array.T)

print(test_sum, test_sum.astype(np.int8))

(2, 1)
[[ 127. -127.]
 [-127.  127.]] [[ 127 -127]
 [-127  127]]


In [108]:
# 127 and 128 take the same amount of time
for i in range(1, 127):
    print(i, 127//i)

1 127
2 63
3 42
4 31
5 25
6 21
7 18
8 15
9 14
10 12
11 11
12 10
13 9
14 9
15 8
16 7
17 7
18 7
19 6
20 6
21 6
22 5
23 5
24 5
25 5
26 4
27 4
28 4
29 4
30 4
31 4
32 3
33 3
34 3
35 3
36 3
37 3
38 3
39 3
40 3
41 3
42 3
43 2
44 2
45 2
46 2
47 2
48 2
49 2
50 2
51 2
52 2
53 2
54 2
55 2
56 2
57 2
58 2
59 2
60 2
61 2
62 2
63 2
64 1
65 1
66 1
67 1
68 1
69 1
70 1
71 1
72 1
73 1
74 1
75 1
76 1
77 1
78 1
79 1
80 1
81 1
82 1
83 1
84 1
85 1
86 1
87 1
88 1
89 1
90 1
91 1
92 1
93 1
94 1
95 1
96 1
97 1
98 1
99 1
100 1
101 1
102 1
103 1
104 1
105 1
106 1
107 1
108 1
109 1
110 1
111 1
112 1
113 1
114 1
115 1
116 1
117 1
118 1
119 1
120 1
121 1
122 1
123 1
124 1
125 1
126 1


So we lose accuracy above 64 as a count of 65 and 128 are both multiplied by 1.

## Reconstruction of Covariance Unit

Things to remember:
* We have a running total - we need to scale by stage_counter.
    * Normally you'd divide by 127 (if max/min are -127/127) so if we have 39 iterations we'd divide by 39.
    * But we're assuming we always have 8bit so we need to divide by 39 then times by 127.
    * We can approximate by determining the nearest power of 2 to the count (i.e. below the count)

In [83]:

class CovarianceUnit:
    """Variation where the mean is assumed to be 0."""

    def __init__(self, size):
        """Initialise.

        Args:
            size: integer setting the 1D size of an input.
        """
        self.size = size
        # Set max value for signed int
        self.max_value = 126
        self.stages = 8
        # Initialise Square Sums
        self.square_sum = np.zeros(
            shape=(size, size, self.stages), dtype=np.int8
        )
        # Define counter for each stage
        self.stage_counter = np.zeros(self.stages, dtype=np.uint8)

    def update(self, data_array):
        """Add a data array to the covariance data.

        This will involve a recursive check.

        Args:
            data_array is a 1D numpy array of length 'size'.
        """
        # Increment current stage counter
        self.stage_counter[0] += 1
        # Add square of input array
        self.square_sum[:, :, 0] += np.dot(data_array, data_array.T)
        self.recursive_update(0)

    def recursive_update(self, i):
        """Update with recursive method.

        Args:
            i - stage to update - integer.
        """
        # Check i is within range
        if i > self.stages:
            return
        # If i is within range check counter
        if self.stage_counter[i] >= 254:
            # Right bit shift the current sum and the next stage sum
            current_stage = self.square_sum[:, :, i]//2
            next_stage = self.square_sum[:, :, i+1]//2
            # Subtract estimate from next stage
            square_sum_dash = current_stage - next_stage
            # Clip at -64 and 63 then times by 2 - returns to int8 space
            square_sum_dash = np.clip(square_sum_dash, -63, 63)*2
            # PBT self.square_sum[:, :, 0] - BUT WILL THIS BE TERNARY?
            rand_ints = np.random.randint(
                self.max_value, size=(self.size, self.size)
            )
            signs = np.sign(square_sum_dash)
            pbt_output = np.where(np.abs(square_sum_dash) > rand_ints, 1, 0)
            # Add to next stage (with signs returned)
            self.square_sum[:, :, i+1] += pbt_output*signs
            # Reset the previous counter and stage
            self.stage_counter[i] = 0
            self.square_sum[:, :, i] = 0 # Is this right as we set it initially to -128
            # Increment next stage counter
            self.stage_counter[i+1] += 1
            self.recursive_update(i+1)
        else:
            return

    @property
    def covariance(self):
        """Compute covariance when requested."""
        # Reconstruct from square sums
        pass



# Re Test

Now we apply the previous tests on the edited classes above.

We might need to rejig the tests a little.

In [71]:
"""Statistical test that cov unit is determining the covariance."""
# Generate random positive definite matrix
cov = np.random.randn(3, 3)
cov = np.dot(cov, cov.T)
cov = cov / cov.max()
# Generate desired mean
mean = np.random.randn(3, 1)

In [72]:
cov, mean

(array([[ 1.        ,  0.37863808,  0.34018587],
        [ 0.37863808,  0.60824758, -0.49372465],
        [ 0.34018587, -0.49372465,  0.9502809 ]]), array([[-2.15622656],
        [ 0.8393449 ],
        [-3.53983336]]))

In [73]:
(cov*127).astype(np.int8), (mean*127).astype(np.int8)

(array([[127,  48,  43],
        [ 48,  77, -62],
        [ 43, -62, 120]], dtype=int8), array([[-17],
        [106],
        [ 63]], dtype=int8))

In [74]:
new_cov = (cov*127).astype(np.int8); new_mean = (mean*127).astype(np.int8)

In [78]:
# Use Cholesky decomposition to get L
L = np.linalg.cholesky(new_cov); L

array([[ 11.26942767,   0.        ,   0.        ],
       [  4.25931125,   7.67191421,   0.        ],
       [  3.81563299, -10.19979712,   1.18536223]])

In [81]:
(np.dot(L, np.random.randn(3, 1)) + new_mean).astype(np.int8)

array([[-23],
       [111],
       [ 50]], dtype=int8)

In [84]:
cov_unit = CovarianceUnit(3)
for _ in range(0, 10000):
    sample = np.dot(L, np.random.randn(3, 1)) + mean
    cov_unit.update(sample.astype(np.int8))

In [88]:
cov_unit.square_sum[:, :, 0], cov_unit.square_sum[:, :, 1]

(array([[   7, -102,   70],
        [-102,   55,  -95],
        [  70,  -95, -102]], dtype=int8), array([[ 0,  6,  3],
        [ 1, -1,  0],
        [ 7,  3, -1]], dtype=int8))

In [89]:
cov_unit.stage_counter

array([55, 39,  0,  0,  0,  0,  0,  0], dtype=uint8)

In [86]:
cov, cov_unit.covariance

(array([[ 1.        ,  0.37863808,  0.34018587],
        [ 0.37863808,  0.60824758, -0.49372465],
        [ 0.34018587, -0.49372465,  0.9502809 ]]), None)

In [ ]:
# Check within 10%
assert np.allclose(cov, cov_unit.covariance, rtol=0.10, atol=0.05)

---
### Test accuracy of covariance unit

In [213]:
class CovarianceUnit:
    """Variation where the mean is assumed to be 0."""

    def __init__(self, size):
        """Initialise.

        Args:
            size: integer setting the 1D size of an input.
        """
        self.size = size
        # Set max value for signed int
        self.max_value = 127
        self.stages = 8
        # Initialise Square Sums
        self.square_sum = np.zeros(
            shape=(size, size, self.stages), dtype=np.int8
        )
        # Initialise Store for last full values
        self.complete = np.zeros(
            shape=(size, size, self.stages), dtype=np.int8
        )
        # Define counter for each stage
        self.stage_counter = np.zeros(self.stages, dtype=np.uint8)

    def update(self, data_array):
        """Add a data array to the covariance data.

        This will involve a recursive check.

        Args:
            data_array is a 1D numpy array of length 'size'.
        """
        # Increment current stage counter
        self.stage_counter[0] += 1
        # Add square of input array
        self.square_sum[:, :, 0] += np.dot(data_array, data_array.T)
        self.recursive_update(0)

    def recursive_update(self, i):
        """Update with recursive method.

        Args:
            i - stage to update - integer.
        """
        # Check i is within range
        if i > self.stages:
            return
        # If i is within range check counter
        if self.stage_counter[i] >= self.max_value:
            # Add to completed estimate
            self.complete[:, :, i] = self.square_sum[:, :, i]
            # PBT
            # Get random integers - from 0 (inclusive) to 128 (exclusive)
            rand_ints = np.random.randint(
                low=0,
                high=self.max_value+1, 
                size=(self.size, self.size)
            )
            signs = np.sign(self.square_sum[:, :, i])
            pbt_output = np.where(np.abs(self.square_sum[:, :, i]) > rand_ints, 1, 0)
            # Add to next stage (with signs returned)
            self.square_sum[:, :, i+1] += pbt_output*signs
            # Reset the previous counter and stage
            self.stage_counter[i] = 0
            self.square_sum[:, :, i] = 0
            # Increment next stage counter
            self.stage_counter[i+1] += 1
            self.recursive_update(i+1)
        else:
            return

    @property
    def covariance(self):
        """Compute covariance when requested."""
        # Reconstruct from square sums
        pass



SyntaxError: invalid syntax (<ipython-input-213-10648d2a21c8>, line 56)

In [214]:
"""Statistical test that cov unit is determining the covariance."""
# Generate random positive definite matrix
cov = np.random.randn(3, 3)
cov = np.dot(cov, cov.T)
cov = cov / cov.max()
new_cov = (cov*127).astype(np.int8)

# Use Cholesky decomposition to get L
L = np.linalg.cholesky(new_cov); print(L, new_cov, sep="\n")

[[ 7.87400787  0.          0.        ]
 [-9.01700902  4.96926035  0.        ]
 [ 9.27100927 -3.09970191  5.60715928]]
[[ 62 -71  73]
 [-71 106 -99]
 [ 73 -99 127]]


In [215]:
np.dot(L, L.T)

array([[ 62., -71.,  73.],
       [-71., 106., -99.],
       [ 73., -99., 127.]])

In [225]:
# Use Cholesky decomposition to get L
L = np.linalg.cholesky(cov); print(L, cov, L*127, sep="\n")

[[ 0.7041427   0.          0.        ]
 [-0.79862943  0.44876818  0.        ]
 [ 0.82129685 -0.29207359  0.49006581]]
[[ 0.49581694 -0.56234908  0.57831018]
 [-0.56234908  0.83920184 -0.78698517]
 [ 0.57831018 -0.78698517  1.        ]]
[[  89.42612275    0.            0.        ]
 [-101.42593715   56.99355894    0.        ]
 [ 104.30470026  -37.09334554   62.23835811]]


In [216]:
cov_unit = CovarianceUnit(3)
for _ in range(0, 127**3+10000):
    sample = np.dot(L, np.random.uniform(low=-1, size=(3, 1)))
    cov_unit.update(sample.astype(np.int8))
cov_unit.square_sum

array([[[ 45,  -9,   0,   0,   0,   0,   0,   0],
        [-20,  -5,   0,   0,   0,   0,   0,   0],
        [-28,   8,   0,   0,   0,   0,   0,   0]],

       [[-20,  -2,   0,   0,   0,   0,   0,   0],
        [-34,   4,   0,   0,   0,   0,   0,   0],
        [-45,   1,   0,   0,   0,   0,   0,   0]],

       [[-28,  -3,   0,   0,   0,   0,   0,   0],
        [-45,   0,   0,   0,   0,   0,   0,   0],
        [ 76,   5,   0,   0,   0,   0,   0,   0]]], dtype=int8)

In [217]:
print(new_cov, cov_unit.square_sum[:, :, 0], cov_unit.square_sum[:, :, 1], sep="\n")

[[ 62 -71  73]
 [-71 106 -99]
 [ 73 -99 127]]
[[ 45 -20 -28]
 [-20 -34 -45]
 [-28 -45  76]]
[[-9 -5  8]
 [-2  4  1]
 [-3  0  5]]


In [218]:
print(new_cov, cov_unit.complete[:, :, 0], cov_unit.complete[:, :, 1], cov_unit.complete[:, :, 2], sep="\n\n")

[[ 62 -71  73]
 [-71 106 -99]
 [ 73 -99 127]]

[[ -87   -2 -128]
 [  -2   -9  -87]
 [-128  -87   28]]

[[-3  8 15]
 [ 7  0 -1]
 [16 11 -3]]

[[-3  2  3]
 [ 1  1  0]
 [ 2 -4  2]]


These don't really match - and the values decrease in size with each layer.

In [219]:
print(sample)

[[-0.27100385]
 [ 2.61107786]
 [-4.22616834]]


In [220]:
np.dot(L, np.ones(shape=(3, 1)))

array([[ 7.87400787],
       [-4.04774867],
       [11.77846663]])

In [221]:
np.dot(L, -1*np.ones(shape=(3, 1)))

array([[ -7.87400787],
       [  4.04774867],
       [-11.77846663]])

In [222]:
np.dot(L, np.random.uniform(low=-1, size=(3, 1)))

array([[-6.22160635],
       [ 9.93553367],
       [-6.14411285]])

Ah this is why this isn't working - our input samples are not ternary. The max is L\*1s and min is L\*-1s. So we can divide by this? Then PBT with uniform numbers. (We still get some numbers slightly bigger than 1 but this should be roughly right?)

In [224]:
sample = np.dot(L, np.random.uniform(low=-1, high=1, size=(3, 1))) / np.dot(L, np.ones(shape=(3, 1)))
print(sample)

rand_vals = np.random.uniform(size=(3,1))
signs = np.sign(sample)
print(signs, rand_vals, sep="\n")
ternary = np.where(np.abs(sample) > rand_vals, 1, 0)
ternary = ternary*signs
print(ternary)

[[-6.99564882e-04]
 [ 1.05159113e+00]
 [-1.11060303e-01]]
[[-1.]
 [ 1.]
 [-1.]]
[[0.09915078]
 [0.25767448]
 [0.67441018]]
[[-0.]
 [ 1.]
 [-0.]]


In [285]:
np.dot(L, np.random.uniform(low=-1, size=(3, 1)))

array([[ 0.10170289],
       [ 0.31597568],
       [-0.28238656]])

We do need to sample from -1 to 1 otherwise we just get the same positive correlation. Low = -1 allows us to flip the bit. **We don't need the scaling.***

In [ ]:
sq_sum = np.zeros(shape=(3,3))
x_sum = np.zeros(shape=(3,1))

In [296]:
for _ in range(0, 127**3+10):
    sample = np.dot(L, np.random.uniform(low=-1, size=(3, 1))) 

    rand_vals = np.random.uniform(size=(3,1))
    signs = np.sign(sample)
    ternary = np.where(np.abs(sample) > rand_vals, 1, 0)
    ternary = ternary*signs
    ternary = ternary.astype(np.int8)
    # print(ternary)
    cov_unit.update(ternary)
    x_sum += ternary
    sq_sum += np.dot(ternary, ternary.T)

print(new_cov, cov_unit.square_sum[:, :, 0], cov_unit.square_sum[:, :, 1], cov_unit.square_sum[:, :, 2], sep="\n\n")
print(new_cov, cov_unit.complete[:, :, 0], cov_unit.complete[:, :, 1], cov_unit.complete[:, :, 2], sep="\n\n")

[[ 62 -71  73]
 [-71 106 -99]
 [ 73 -99 127]]

[[ 24 -13  18]
 [-13  30 -19]
 [ 18 -19  35]]

[[ 29 -20  14]
 [-17  40 -27]
 [ 15 -20  35]]

[[ 1 -1  1]
 [-3  4 -1]
 [ 2  0  3]]
[[ 62 -71  73]
 [-71 106 -99]
 [ 73 -99 127]]

[[ 42 -22  22]
 [-22  52 -29]
 [ 22 -29  56]]

[[ 45 -20  20]
 [-21  50 -28]
 [ 25 -27  49]]

[[ 47 -29  23]
 [-24  55 -30]
 [ 23 -30  50]]


In [295]:
print((sq_sum/(127**3+10))*127, (x_sum/127**3+10)*127)

[[ 45.7703141  -24.05872067  24.17391633]
 [-24.05872067  56.94516189 -32.34350879]
 [ 24.17391633 -32.34350879  60.61152132]] [[1269.98604997]
 [1270.0486701 ]
 [1269.98995598]]


Ah now we are getting close. Large values seem reduced? 127 > 58/59 But accurate representation of the square sum.

We seem to be missing negative values from stages 1+. Ah here the sq_sum is +ve. Because both signs being the same will give +ve correlation - we seem to be missing -ve correlation.

Maybe go back to a simpler test of just feeding in a constant value? Same and different?

Ah - does our input need to be between 0 and 1? Does -ve input cause issues? No because if we remove the mean we should get negative inputs. 

Is the problem that we haven't got a zero mean? Despite assuming a zero mean.

Is it something to do with the scaling - the dividing of L? Yes.

Without this the signs work but the values are about 50% off.

In [239]:
def rand_same(size=2):
    a = np.empty([size, 1])
    a.fill(np.random.randint(2))
    return a

def rand_diff(size=2):
    a = np.zeros([size, 1])
    index = np.random.randint(size)
    a[index] = 1
    return a

In [249]:
print(rand_same(size=3), rand_diff(size=3))

[[0.]
 [0.]
 [0.]] [[0.]
 [0.]
 [1.]]


In [231]:
print(self.square_sum[:, :, 0])

[[32 30 16]
 [30 51 25]
 [16 25 28]]


In [232]:
self = cov_unit
rand_ints = np.random.randint(
                low=0,
                high=self.max_value+1, 
                size=(self.size, self.size)
            )
signs = np.sign(self.square_sum[:, :, 0])
pbt_output = np.where(np.abs(self.square_sum[:, :, 0]) > rand_ints, 1, 0)
# Add to next stage (with signs returned)
print(rand_ints, signs, pbt_output, pbt_output*signs)

[[22 89 33]
 [ 6 83  3]
 [99 70 52]] [[1 1 1]
 [1 1 1]
 [1 1 1]] [[1 0 0]
 [1 0 1]
 [0 0 0]] [[1 0 0]
 [1 0 1]
 [0 0 0]]


In [235]:
self = cov_unit
cov_sum = np.zeros(shape=(3,3))
for _ in range(0, 127):
    rand_ints = np.random.randint(
                low=0,
                high=self.max_value+1, 
                size=(self.size, self.size)
            )
    signs = np.sign(new_cov)
    pbt_output = np.where(np.abs(new_cov) > rand_ints, 1, 0)
    cov_sum += pbt_output*signs
    # Add to next stage (with signs returned)
print(new_cov, rand_ints, signs, pbt_output, pbt_output*signs, cov_sum, sep= "\n")

[[ 62 -71  73]
 [-71 106 -99]
 [ 73 -99 127]]
[[ 73  33  81]
 [ 10  46 117]
 [ 11  18  62]]
[[ 1 -1  1]
 [-1  1 -1]
 [ 1 -1  1]]
[[0 1 0]
 [1 1 0]
 [1 1 1]]
[[ 0 -1  0]
 [-1  1  0]
 [ 1 -1  1]]
[[  69.  -77.   66.]
 [ -69.  107. -107.]
 [  83. -103.  127.]]


So the general PBT sum over consecutive samples does work if we start from the real cov.

In [227]:
ternary

array([[-1],
       [-1],
       [ 0]], dtype=int8)

In [199]:
cov_unit.complete[:, :, 2]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int8)

Ah - when we iterate we reset the values

In [114]:
cov_unit.stage_counter

array([94, 78,  1,  0,  0,  0,  0,  0], dtype=uint8)

In [124]:
np.nonzero(cov_unit.stage_counter)

(array([0, 1, 2]),)

---

# Power Iteration

We need to look at this next because we need to see how to return the covariance.

For power iterator we have the following data:
* ev - this is normally a float numpy 1D array with fractional values
* cov - this is normally a float numpy 2D array 
* scaler - this will have a fractional value greater than 1 (normally greater than 1.4).

We can keep ev and cov in signed 8 bit space and take the 1/127 out of the calculations (putting them at the front).

We have:
* cov^power \* ev - we can take out (1/127)^2 if power = 1.
* ev^T.input_data - we can take out (1/127) - input_data is -1, 0, 1 so so we have the same possible scale at ev but flipped - why we need symmetric scales - clip at -127 to 127.

We can either operate in the next power of 2 space - e.g. 16 bit vs 8 bit - the result is stored in 16 bit space to represent a max of 128 \* 128 (if signed - 255 if unsigned). OR. We can divide both the cov and the ev by 16 - so that the we have max 16\*16 = 256 (right shift by 3). 

In [32]:
cov = 127*np.ones(shape=(2,2), dtype=np.int8); print(cov)

[[127 127]
 [127 127]]


In [34]:
ev = 127*np.ones(shape=(2,1), dtype=np.int8); print(ev)
temp_ev = np.matmul(cov, ev).astype(np.int16); print(temp_ev)

[[127]
 [127]]
[[2]
 [2]]


In [36]:
temp_cov = cov.astype(np.int16)
temp_ev = ev.astype(np.int16)
temp_ev = np.matmul(temp_cov, temp_ev); print(temp_ev)
print(f"Cov dtype={cov.dtype}, ev dtpye={ev.dtype}, temp_cov dtype={temp_cov.dtype}, temp_ev dtype={temp_ev.dtype}")

[[32258]
 [32258]]
Cov dtype=int8, ev dtpye=int8, temp_cov dtype=int16, temp_ev dtype=int16


In [37]:
%%timeit 
t_c = cov.astype(np.int16)

679 ns ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [38]:
127*127+127*127

32258

I.e. max is bit_depth_max^2 * L. So we need to normalise by bit_depth_max\*L

In [41]:
print(temp_ev / (2*127), temp_ev / (2*128))
print(temp_ev // (2*127), temp_ev // (2*128))

[[127.]
 [127.]] [[126.0078125]
 [126.0078125]]
[[127]
 [127]] [[126]
 [126]]


In [47]:
print(np.right_shift(temp_ev, 8), np.right_shift(temp_ev, 8).dtype, np.right_shift(temp_ev, 8).astype(np.int8))

[[126]
 [126]] int16 [[126]
 [126]]


In [45]:
%%timeit
np.right_shift(temp_ev, 8)

1.52 µs ± 28.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [46]:
%%timeit
temp_ev // (2*128)

1.65 µs ± 27.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Pretty much the same.

In [51]:
np.log2(2).astype(np.uint8)

1

In [52]:
cov = 127*np.ones(shape=(3,3), dtype=np.int8); print(cov)
ev = 127*np.ones(shape=(3,1), dtype=np.int8); print(ev)
temp_ev = np.matmul(cov, ev).astype(np.int16); print(temp_ev)
temp_cov = cov.astype(np.int16)
temp_ev = ev.astype(np.int16)
temp_ev = np.matmul(temp_cov, temp_ev); print(temp_ev)
print(f"Cov dtype={cov.dtype}, ev dtpye={ev.dtype}, temp_cov dtype={temp_cov.dtype}, temp_ev dtype={temp_ev.dtype}")

[[127 127 127]
 [127 127 127]
 [127 127 127]]
[[127]
 [127]
 [127]]
[[3]
 [3]
 [3]]
[[-17149]
 [-17149]
 [-17149]]
Cov dtype=int8, ev dtpye=int8, temp_cov dtype=int16, temp_ev dtype=int16


Ah we need to divide ev by L befpre we multiply. (Or right shift if length is a power of 2.)

In [54]:
cov = 127*np.ones(shape=(3,3), dtype=np.int8); print(cov)
ev = 127*np.ones(shape=(3,1), dtype=np.int8); print(ev)
temp_ev = np.matmul(cov, ev).astype(np.int16); print(temp_ev)
temp_cov = cov.astype(np.int16)
temp_ev = ev.astype(np.int16) // 3
temp_ev = np.matmul(temp_cov, temp_ev); print(temp_ev)
print(f"Cov dtype={cov.dtype}, ev dtpye={ev.dtype}, temp_cov dtype={temp_cov.dtype}, temp_ev dtype={temp_ev.dtype}")

[[127 127 127]
 [127 127 127]
 [127 127 127]]
[[127]
 [127]
 [127]]
[[3]
 [3]
 [3]]
[[16002]
 [16002]
 [16002]]
Cov dtype=int8, ev dtpye=int8, temp_cov dtype=int16, temp_ev dtype=int16


In [55]:
cov = 127*np.ones(shape=(2,2), dtype=np.int8); print(cov)
ev = 127*np.ones(shape=(2,1), dtype=np.int8); print(ev)
temp_ev = np.matmul(cov, ev).astype(np.int16); print(temp_ev)
temp_cov = cov.astype(np.int16)
temp_ev = ev.astype(np.int16) // 2
temp_ev = np.matmul(temp_cov, temp_ev); print(temp_ev)
print(f"Cov dtype={cov.dtype}, ev dtpye={ev.dtype}, temp_cov dtype={temp_cov.dtype}, temp_ev dtype={temp_ev.dtype}")

[[127 127]
 [127 127]]
[[127]
 [127]]
[[2]
 [2]]
[[16002]
 [16002]]
Cov dtype=int8, ev dtpye=int8, temp_cov dtype=int16, temp_ev dtype=int16


The eigenvalue has a max of bd\*L. The top_1 will have a max of bd_max^2 * L, the top_1 * ev will have a max bd_max^3\*L^2, the bottom will have a max of bd^2\*L.

We can divide ev by L to remove the L term. Then we can divide top_1 and bottom by 127 before the final calculation to get bd and bd then we'll have bd^2 / bd > bd which we can represent at the bit depth.

In [ ]:
class PowerIterator:
    """Module to determine an eigenvector using power iteration.
    
    Operates on 8-bit values. The right shift can be replaced for "// length" to allow all lengths
    """

    def __init__(self, length):
        """Initialise.

        Args:
            length: integer setting the 1D size of the eigenvector 
            - needs to be a power of 2.
        """
        # Store shift values for later bit shifting - np.int8 has max 2^7-1
        self.shift_val = np.log2(length).astype(np.uint8) # This only works for lengths that are powers of 2
        self.ev = np.zeros(shape=(length,1), dtype=np.int8)
        # Initialise eigenvector as random vector
        # Loop if we get all zeros
        while not self.ev.any():
            # Set range to 8 bit signed -127 to 127 (to be symmetrical)
            self.ev = np.random.randint(255, size=(length, 1))-127
        # Define placeholder for covariance matrix
        self.cov = np.zeros(shape=(length, length), dtype=np.int8)
        # Define eigenvalue
        self.rayleigh = np.zeros(1, dtype=np.int8)

    def iterate(self):
        """One pass of iteration.
        
        Applies power iteration with power = 1.
        Casts cov and ev to 16-bit, matmuls then casts
        back to 8-bit after scaling.
        """
        # Check cov is not all zero - if all 0 you get nan
        if self.cov.any():
            # Convert cov and ev to 16-bit
            temp_cov = self.cov.astype(np.int16)
            # Also shift right to divide by L
            temp_ev = np.right_shift(self.ev.astype(np.int16), self.shift_val)
            # Just do one multiplication per round
            temp_ev = np.matmul(temp_cov, temp_ev)
            # Right shift to convert back to 8-bit
            # We need to shift by bit_depth_max + length
            temp_ev = np.right_shift(temp_ev, 7).astype(np.int8)
            self.ev = temp_ev
        return temp_ev

    @property
    def eigenvector(self):
        """Return the top eigenvector."""
        return self.ev.copy()

    @property
    def eigenvalue(self):
        """Return associated eigenvalue."""
        # Convert cov and ev to 16-bit
        temp_cov = self.cov.astype(np.int16)
        # Also shift right to divide by L
        temp_ev = np.right_shift(self.ev.astype(np.int16), self.shift_val)
        # Compute in 16-bit space 
        top_1 = np.matmul(temp_ev.T, temp_cov)
        bottom = np.matmul(temp_ev.T, temp_ev)
        # Scale both to divide by bit_depth_max (i.e. 128 for 8bit)
        top_1 = np.right_shift(top_1, 7)
        bottom = np.right_shift(bottom, 7)
        # Compute eigenvalue as Raleigh Quotient
        rayleigh = np.matmul(top_1, temp_ev) / bottom
        rayleigh = rayleigh.astype(np.int8)
        self.rayleigh = rayleigh
        return rayleigh

    def load_covariance(self, cov):
        """Update the covariance matrix."""
        self.cov = cov
        # Put here an update of an existing matrix?


---
# Reference Stuff

In [ ]:
# Version with variable bit depth
class CovarianceUnit:
    """Variation where the mean is assumed to be 0."""
    
    def __init__(self, size, bit_depth=8):
        """Initialise.

        Args:
            size: integer setting the 1D size of an input.
            bit_depth: number of bits to store representations (multiple of 8) - max 64
        """
        self.size = size
        self.bit_depth = bit_depth
        self.max_value = 2**bit_depth
        # Set bit_depth modulo 8
        if bit_depth > 64: bit_depth = 64
        if bit_depth < 8: bit_depth = 8
        stages = 64//bit_depth
        # Set 
        # Initialise Square Sum
        self.square_sum = np.ones(shape=(size, size, stages), dtype=)
        # Set to halfway between 0 and bit depth (128 for 8-bit)
        self.square_sum = 2**(bit_depth-1)*self.square_sum
        # Define counter for each stage
        self.stage_counter = np.zeros(stages)
        
    def update(self, x):
        """Add a data point x.
        
        This will involve a recursive check.

        x is a 1D numpy array of length 'size'.
        """
        self.stage_counters[0] += 1
        self.square_sum[:, :, 0] += np.dot(x, x.T)
        # If the count reaches a threshold, can we divide the sums
        if self.stage_counters[0] >= (2**bit_depth)-1:
            # Subtract estimate from next stage
            square_sum_dash = self.square_sum[:, :, 0] - self.square_sum[:, :, 1]
            # PBT self.square_sum[:, :, 0] - BUT WILL THIS BE TERNARY? 
            rand_ints = np.random.randint(
                self.max_value, size=(self.size, self.size)
            )
            signs = np.signs(square_sum_dash)
            pbt_output = np.where(self.square_sum[:, :, 0], rand_ints, 1, 0)
            # Add to next stage
            self.square_sum[:, :, 1] += pbt_output
            # Increment next stage counter
            self.stage_counters[1] += 1
            # Reset the previous counter
            self.stage_counters[0] = 0
            if self.stage_counters[1] >= (2**bit_depth)-1:
                

    @property
    def covariance(self):
        """Compute covariance when requested."""
        # Only return is count is 1 or more
        if self.count:
            cov = self.square_sum / self.count
        else:
            cov = self.square_sum
        return cov
